imports

In [19]:
import pandas as pd

writer = pd.ExcelWriter('./results/data_of_all.xlsx', engine='xlsxwriter')

config

In [20]:
config_prefix = ['BR', 'BI', 'TR', 'TI']

config_datasheet = r'./results/preprocessed.xlsx'

import df

In [21]:
raw = pd.read_excel(config_datasheet)
df = pd.DataFrame(raw)
df["Flag"] = df["Flag"].apply(str)

remove outliers

In [22]:
#split dataframes into dataframes for the 2*2 related factors
df_BU_R = df.loc[(df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="BU")].drop("ClickData", axis=1)
df_BU_I = df.loc[(df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="BU")].drop("ClickData", axis=1)
df_TD_R = df.loc[(df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="TD")].drop("ClickData", axis=1)
df_TD_I = df.loc[(df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="TD")].drop("ClickData", axis=1)

In [23]:
df_array_independent = [df_BU_R, df_BU_I, df_TD_R, df_TD_I]
df_means = [float(df["ResponseTime"].mean()) for df in df_array_independent]
df_stds = [float(df["ResponseTime"].std()) for df in df_array_independent]

for idx, df in enumerate(df_array_independent):
    mean = float(df_means[idx])
    stddev = float(df_stds[idx])
    for row_idx, row in df.iterrows():
        response_time = row["ResponseTime"]
        if mean+1.5*stddev < response_time < mean+2*stddev:
            df.loc[row_idx, "Flag"] = "notNormal"
            
        if response_time <= 10 and row["Correctness"] == False or response_time >= mean+2*stddev:
            df.loc[row_idx, "Flag"] = "Outlier"

df = df_BU_R
df = df.append(df_BU_I)
df = df.append(df_TD_R)
df = df.append(df_TD_I)
df.to_excel(writer, "labeled")

for idx, df in enumerate(df_array_independent):
    df.loc[df["Flag"]=="Outlier", "ResponseTime"] = df_means[idx]

In [24]:
for df_idx, _df in enumerate(df_array_independent):
    for row_idx, row in df.iterrows():
        id_value = row["Subject"]
        entries_id_BU_R = df_BU_R.loc[(df_BU_R["Subject"] == id_value) & (df_BU_R["Flag"] == "Outlier")]
        entries_id_BU_I = df_BU_I.loc[(df_BU_I["Subject"] == id_value) & (df_BU_I["Flag"] == "Outlier")] 
        entries_id_TD_R = df_TD_R.loc[(df_TD_R["Subject"] == id_value) & (df_TD_R["Flag"] == "Outlier")]
        entries_id_TD_I = df_TD_I.loc[(df_TD_I["Subject"] == id_value) & (df_TD_I["Flag"] == "Outlier")]
        
        df_per_id = entries_id_BU_R
        df_per_id = df_per_id.append(entries_id_BU_I)
        df_per_id = df_per_id.append(entries_id_TD_R)
        df_per_id = df_per_id.append(entries_id_TD_I)
        
        number_of_outliers_per_subject = len(df_per_id)
        
        if number_of_outliers_per_subject == 2:
            
            df_nn_BU_R = df_BU_R.loc[(df_BU_R["Subject"] == id_value) & (df_BU_R["Flag"] == "notNormal")]
            df_nn_BU_I = df_BU_I.loc[(df_BU_I["Subject"] == id_value) & (df_BU_I["Flag"] == "notNormal")]
            df_nn_TD_R = df_TD_R.loc[(df_TD_R["Subject"] == id_value) & (df_TD_R["Flag"] == "notNormal")]
            df_nn_TD_I = df_TD_I.loc[(df_TD_I["Subject"] == id_value) & (df_TD_I["Flag"] == "notNormal")]
            
            num_non_normals = len(df_nn_BU_R) + len(df_nn_BU_I) + len(df_nn_TD_R) + len(df_nn_TD_I)
            
            if num_non_normals > 0:
                print("Participant with the following number got deleted: " + str(id_value) + ". Because 2 where missing and " + str(num_non_normals) + " datapoints where not normal.")
                df_BU_R = df_BU_R.loc[df_BU_R["Subject"]!=id_value]
                df_BU_I = df_BU_I.loc[df_BU_I["Subject"]!=id_value]
                df_TD_R = df_TD_R.loc[df_TD_R["Subject"]!=id_value]
                df_TD_I = df_TD_I.loc[df_TD_I["Subject"]!=id_value]
                
        if number_of_outliers_per_subject > 2:
            print("Participant with the following number got deleted: " + str(id_value) + ". Because " + str(number_of_outliers_per_subject) + " where missing.")
            df_BU_R = df_BU_R.loc[df_BU_R["Subject"]!=id_value]
            df_BU_I = df_BU_I.loc[df_BU_I["Subject"]!=id_value]
            df_TD_R = df_TD_R.loc[df_TD_R["Subject"]!=id_value]
            df_TD_I = df_TD_I.loc[df_TD_I["Subject"]!=id_value]
                
df_array_independent = [df_BU_R, df_BU_I, df_TD_R, df_TD_I]  

Participant with the following number got deleted: 842. Because 3 where missing.
Participant with the following number got deleted: 712. Because 4 where missing.
Participant with the following number got deleted: 942. Because 2 where missing and 1 datapoints where not normal.
Participant with the following number got deleted: 785. Because 3 where missing.


In [25]:
algo_name_array = [name for name in df["Algorithm"].unique()]

split dataframes 

In [26]:
df_cat_array = df_array_independent

df = df_array_independent[0]
df = df.append(df_array_independent[1])
df = df.append(df_array_independent[2])
df = df.append(df_array_independent[3])

df_algo_cat_array = []
for algo_name in algo_name_array:
    tmp_df = []
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="BU")])
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="BU")])
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="TD")])
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="TD")])
    df_algo_cat_array.append(tmp_df)

create result

In [29]:
cols = ["Algorithm", "Comprehension",
        "R_ResponseTimeMean", "R_ResponseTimeDeviation",
        "I_ResponseTimeMean", "I_ResponseTimeDeviation"]

result_df = pd.DataFrame(columns=cols)

count_cols = ["Algorithm", "Comprehension", "Programming", "Right", "Total"]
count_df = pd.DataFrame(columns=count_cols)

for algo_df_array in df_algo_cat_array:
    for i in range(2):
        rec =  algo_df_array[i*2+0]
        it =  algo_df_array[i*2+1]
        
        Algorithm = rec.iloc[0]["Algorithm"]
        Comprehension = "Bottom-Up"
        if i == 1:
            Comprehension = "Top-Down"
            
        count_df = count_df.append(pd.DataFrame([[Algorithm, Comprehension, "R", len(rec.loc[rec["Correctness"]==True]) ,len(rec)]], columns=count_cols))
        count_df = count_df.append(pd.DataFrame([[Algorithm, Comprehension, "I", len(it.loc[it["Correctness"]==True]) ,len(it)]], columns=count_cols))
        
        R_ResponseTimeMean = int(rec["ResponseTime"].mean())
        R_ResponseTimeDeviation = int(rec["ResponseTime"].std())
        
        I_ResponseTimeMean = int(it["ResponseTime"].mean())
        I_ResponseTimeDeviation = int(it["ResponseTime"].std())
        
        data = [
            Algorithm, Comprehension,
            R_ResponseTimeMean, R_ResponseTimeDeviation,
            I_ResponseTimeMean, I_ResponseTimeDeviation,
        ]
        
        result_df = result_df.append(pd.DataFrame([data], columns=cols))
        
for i in range(2):
    rec =  df_cat_array[i*2+0]
    it =  df_cat_array[i*2+1]
    
    Algorithm = "Total"
    Comprehension = "Top-Down"
    if i == 1:
        Comprehension = "Bottom-Up"
    
    R_ResponseTimeMean = int(rec["ResponseTime"].mean())
    R_ResponseTimeDeviation = int(rec["ResponseTime"].std())
    
    I_ResponseTimeMean = int(it["ResponseTime"].mean())
    I_ResponseTimeDeviation = int(it["ResponseTime"].std())
    
    data = [
        Algorithm, Comprehension,
        R_ResponseTimeMean, R_ResponseTimeDeviation,
        I_ResponseTimeMean, I_ResponseTimeDeviation,
    ]
    
    result_df = result_df.append(pd.DataFrame([data], columns=cols))

print("Write total means to excel sheet")
result_df.to_excel(writer, sheet_name="corrects.info")
result_df

Write total means to excel sheet


,Algorithm,Comprehension,R_ResponseTimeMean,R_ResponseTimeDeviation,I_ResponseTimeMean,I_ResponseTimeDeviation
0,BinarySearch,Bottom-Up,368,157,187,145
0,BinarySearch,Top-Down,226,126,211,158
0,BubbleSort,Bottom-Up,232,84,204,169
0,BubbleSort,Top-Down,36,39,222,102
0,Factorial,Bottom-Up,93,66,127,81
0,Factorial,Top-Down,78,36,112,101
0,Fibonacci,Bottom-Up,174,136,212,132
0,Fibonacci,Top-Down,161,79,182,111
0,IntegerBinary,Bottom-Up,167,113,224,117
0,IntegerBinary,Top-Down,157,101,207,94


In [31]:
count_df["percentages"] = float(count_df["Right"])/float(count_df["Total"])
count_df

TypeError: cannot convert the series to <class 'float'>

In [16]:
df_algo_cat_array[0][0]

,Subject,ProgrammingStyle,Comprehension,ResponseTime,Algorithm,Flag,Correctness
0,638,R,BU,296.000000,BinarySearch,nan,True
2,750,R,BU,231.515723,BinarySearch,Outlier,True
3,784,R,BU,438.000000,BinarySearch,nan,True
4,829,R,BU,449.000000,BinarySearch,nan,False
5,841,R,BU,123.000000,BinarySearch,nan,True
6,857,R,BU,467.000000,BinarySearch,nan,True
7,938,R,BU,575.000000,BinarySearch,nan,True


create means of means for total

In [21]:
cols = ["Algorithm", "Comprehension",
        "R_ResponseTimeMean", "R_ResponseTimeDeviation",
        "I_ResponseTimeMean", "I_ResponseTimeDeviation"]

means_per_algo = pd.DataFrame(columns=cols)

for algo_df_array in df_algo_cat_array:
    for i in range(2):
        rec =  algo_df_array[i*2+0].loc[algo_df_array[i*2+0]["Flag"]!="Outlier"]
        it =  algo_df_array[i*2+1].loc[algo_df_array[i*2+1]["Flag"]!="Outlier"]
        
        Algorithm = rec.iloc[0]["Algorithm"]
        Comprehension = "Top-Down"
        if i == 1:
            Comprehension = "Bottom-Up"
        
        R_ResponseTimeMean = int(rec["ResponseTime"].mean())
        R_ResponseTimeDeviation = int(rec["ResponseTime"].std())
        
        I_ResponseTimeMean = int(it["ResponseTime"].mean())
        I_ResponseTimeDeviation = int(it["ResponseTime"].std())
        
        data = [
            Algorithm, Comprehension,
            R_ResponseTimeMean, R_ResponseTimeDeviation,
            I_ResponseTimeMean, I_ResponseTimeDeviation,
        ]
        
        means_per_algo = means_per_algo.append(pd.DataFrame([data], columns=cols))

df_top_down = means_per_algo.loc[means_per_algo["Comprehension"] == "Top-Down"]
df_bottom_up = means_per_algo.loc[means_per_algo["Comprehension"] == "Bottom-Up"]

In [22]:
means_bu_r = int(df_bottom_up["R_ResponseTimeMean"].mean())
means_bu_i = int(df_bottom_up["I_ResponseTimeMean"].mean())
means_td_r = int(df_top_down["R_ResponseTimeMean"].mean())
means_td_i = int(df_top_down["I_ResponseTimeMean"].mean())

stddev_bu_r = int(df_bottom_up["R_ResponseTimeMean"].std())
stddev_bu_i = int(df_bottom_up["I_ResponseTimeMean"].std())
stddev_td_r = int(df_top_down["R_ResponseTimeMean"].std())
stddev_td_i = int(df_top_down["I_ResponseTimeMean"].std())

df = pd.DataFrame([], columns=cols)
df = df.append(pd.DataFrame([["Total", "Top-Down", means_td_r, stddev_td_r, means_td_i, stddev_td_i]], columns=cols))
df = df.append(pd.DataFrame([["Total", "Bottom-Up", means_bu_r, stddev_bu_r, means_bu_i, stddev_bu_i]], columns=cols))
means_per_algo = means_per_algo.append(df)
means_per_algo.to_excel(writer, sheet_name="corrects.weighted_info")
display(means_per_algo)
writer.save()

,Algorithm,Comprehension,R_ResponseTimeMean,R_ResponseTimeDeviation,I_ResponseTimeMean,I_ResponseTimeDeviation
0,BinarySearch,Top-Down,391,158,187,145
0,BinarySearch,Bottom-Up,226,126,209,163
0,BubbleSort,Top-Down,232,86,204,169
0,BubbleSort,Bottom-Up,36,39,220,108
0,Factorial,Top-Down,88,61,127,81
0,Factorial,Bottom-Up,78,36,112,101
0,Fibonacci,Top-Down,171,138,212,132
0,Fibonacci,Bottom-Up,161,79,179,113
0,IntegerBinary,Top-Down,167,113,224,117
0,IntegerBinary,Bottom-Up,157,101,207,94
